### Strategy Explanation

1. **Precomputation of Allowed Tuples:**
   - Use a Sieve of Eratosthenes to compute all 5, 6-digit primes.
   - For each prime, convert it to a tuple of digits and group these tuples by their digit sum. This sum is the magic constant \(A\) used in the square.

2. **Modeling with CP-SAT and Table Constraints:**
   - For each possible magic sum \(A\) (for which there is at least one valid 6-digit prime tuple), set up a CP-SAT model using OR-Tools.
   - Create a N×N grid of digit variables (each between 0 and 9).
   - Add table constraints so that every row, every column, and the two main diagonals (read as N-digit numbers) must match one of the precomputed tuples. This ensures that the numbers are prime and have the digit sum \(A\).

3. **Solution Enumeration and Cost Calculation:**
   - A custom callback processes each solution found, constructing the 2N+2 numbers (N rows, N columns, and 2 diagonals).
   - The “cost” of a solution is computed by taking the unique primes from these "2N+2" numbers, counting the frequency of each digit, and summing the incremental cost (0 for the first occurrence, 1 for the second, etc.).
   - Global tracking is used to remember the solutions with the minimal and maximal cost.

4. **Efficiency Considerations:**
   - **Table Constraints:** Restricting each line to only allowed prime tuples drastically prunes the search space.
   - **Magic Sum Loop:** Iterating over only those \(A\) values that yield valid tuples avoids unnecessary models.
   - **Time Limits:** A per-model time limit ensures that the search remains tractable.


In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0



**Solving the 5×5 prime‐magic square problem.**


Each row, column, and the two main diagonals (read as 5–digit numbers with no leading zero)
must be prime, and their digit sum must equal A (the same for all 8 “lines”).
The “cost” is computed from the unique primes that appear in the 12 numbers (5 rows, 5 columns, 2 diagonals):
for each digit d, if it appears n times overall the cost for that digit is 0+1+…+(n–1) = n*(n–1)//2;
the total cost is the sum over digits.
We wish to find one square with minimal cost and one with maximal cost.


In [2]:
from ortools.sat.python import cp_model
import math

# A simple prime–checker.
def is_prime(n):
    if n < 2:
        return False
    if n % 2 == 0:
        return n == 2
    r = int(math.isqrt(n))
    for i in range(3, r + 1, 2):
        if n % i == 0:
            return False
    return True

# Precompute all 5–digit primes (from 10000 to 99999)
# and group them by their digit sum.
def get_allowed_tuples_by_sum():
    allowed = {s: [] for s in range(46)}  # possible sums 0..45
    for n in range(10000, 100000):
        if is_prime(n):
            digits = tuple(int(d) for d in str(n))
            s = sum(digits)
            allowed[s].append(digits)
    return allowed

# Given a 5x5 grid (list of lists of ints), compute the 12 numbers
# (5 rows, 5 columns, 2 diagonals), take the set of unique primes,
# count the digit frequencies, and compute cost = sum(n*(n-1)//2 for each digit).
def compute_cost(grid):
    primes = []
    # Rows:
    for i in range(5):
        num = int("".join(str(grid[i][j]) for j in range(5)))
        primes.append(num)
    # Columns:
    for j in range(5):
        num = int("".join(str(grid[i][j]) for i in range(5)))
        primes.append(num)
    # Diagonals:
    num_diag1 = int("".join(str(grid[i][i]) for i in range(5)))
    num_diag2 = int("".join(str(grid[i][4-i]) for i in range(5)))
    primes.append(num_diag1)
    primes.append(num_diag2)
    unique_primes = set(primes)

    digit_counts = {str(d): 0 for d in range(10)}
    for p in unique_primes:
        for d in str(p):
            digit_counts[d] += 1
    cost = sum(v*(v-1)//2 for v in digit_counts.values())
    return cost, unique_primes, digit_counts

best_min_cost = None
best_min_solution = None
best_min_A = None

best_max_cost = None
best_max_solution = None
best_max_A = None

# A solution callback to examine every solution from a CP-SAT solve.
class GridSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, grid, A_fixed):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._grid = grid  # 5x5 list of cp_model.IntVar
        self._A = A_fixed  # the fixed A value for this model
        self._solution_count = 0

    def on_solution_callback(self):
        global best_min_cost, best_min_solution, best_min_A
        global best_max_cost, best_max_solution, best_max_A

        self._solution_count += 1
        # Extract the grid solution.
        grid_sol = [[self.Value(self._grid[i][j]) for j in range(5)] for i in range(5)]
        cost, primes_set, digit_counts = compute_cost(grid_sol)

        if best_min_cost is None or cost < best_min_cost:
            best_min_cost = cost
            best_min_solution = [row[:] for row in grid_sol]
            best_min_A = self.Value(self._A)

        if best_max_cost is None or cost > best_max_cost:
            best_max_cost = cost
            best_max_solution = [row[:] for row in grid_sol]
            best_max_A = self.Value(self._A)


def main():
    global best_min_cost, best_min_solution, best_min_A
    global best_max_cost, best_max_solution, best_max_A

    allowed_by_sum = get_allowed_tuples_by_sum()

    # For each possible magic sum A for which there are allowed primes, build a model.
    for A_val in range(1, 46):
        if not allowed_by_sum[A_val]:
            continue
        print(f"Trying A = {A_val} with {len(allowed_by_sum[A_val])} allowed tuples")

        model = cp_model.CpModel()

        # Create a 5x5 grid of digit variables (each in 0..9).
        grid = [[model.NewIntVar(0, 9, f"cell_{i}_{j}") for j in range(5)]
                for i in range(5)]

        # Fix A to be A_val.
        A_fixed = model.NewIntVar(A_val, A_val, "A")

        # For each row, force the 5–tuple (left–to–right) to be one of the allowed primes.
        for i in range(5):
            model.AddAllowedAssignments(grid[i], allowed_by_sum[A_val])

        # For each column, gather the 5 digits (top–to–bottom) and add table constraint.
        for j in range(5):
            col = [grid[i][j] for i in range(5)]
            model.AddAllowedAssignments(col, allowed_by_sum[A_val])

        # The two main diagonals:
        diag1 = [grid[i][i] for i in range(5)]
        diag2 = [grid[i][4-i] for i in range(5)]
        model.AddAllowedAssignments(diag1, allowed_by_sum[A_val])
        model.AddAllowedAssignments(diag2, allowed_by_sum[A_val])

        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 30.0
        solution_printer = GridSolutionPrinter(grid, A_fixed)

        status = solver.SearchForAllSolutions(model, solution_printer)
        print(f"A = {A_val} produced {solution_printer._solution_count} solution(s).")

    if best_min_solution is not None:
        print("\nMinimal cost solution:")
        print(f"Cost: {best_min_cost}  (A = {best_min_A})")
        for row in best_min_solution:
            print(" ".join(map(str, row)))
    else:
        print("No solution found for minimal cost.")

    if best_max_solution is not None:
        print("\nMaximal cost solution:")
        print(f"Cost: {best_max_cost}  (A = {best_max_A})")
        for row in best_max_solution:
            print(" ".join(map(str, row)))
    else:
        print("No solution found for maximal cost.")

if __name__ == "__main__":
    main()

Trying A = 4 with 4 allowed tuples
A = 4 produced 0 solution(s).
Trying A = 5 with 12 allowed tuples
A = 5 produced 0 solution(s).
Trying A = 7 with 28 allowed tuples
A = 7 produced 0 solution(s).
Trying A = 8 with 45 allowed tuples
A = 8 produced 0 solution(s).
Trying A = 10 with 95 allowed tuples
A = 10 produced 0 solution(s).
Trying A = 11 with 143 allowed tuples
A = 11 produced 6 solution(s).
Trying A = 13 with 236 allowed tuples
A = 13 produced 20 solution(s).
Trying A = 14 with 272 allowed tuples
A = 14 produced 0 solution(s).
Trying A = 16 with 411 allowed tuples
A = 16 produced 0 solution(s).
Trying A = 17 with 479 allowed tuples
A = 17 produced 238 solution(s).
Trying A = 19 with 630 allowed tuples
A = 19 produced 340 solution(s).
Trying A = 20 with 664 allowed tuples
A = 20 produced 0 solution(s).
Trying A = 22 with 742 allowed tuples
A = 22 produced 0 solution(s).
Trying A = 23 with 757 allowed tuples
A = 23 produced 387 solution(s).
Trying A = 25 with 741 allowed tuples
A =

**Solving the N=6 prime–magic square problem.**


Each row, column, and the two main diagonals (read as 6–digit numbers with no leading zero)
must be prime, and their digit sum must equal the same magic constant A.
The “cost” is computed from the set of unique primes that appear in the 14 numbers
(6 rows, 6 columns, 2 diagonals): for each digit d, if it appears n times overall then
the cost for that digit is 0+1+…+(n–1) = n*(n–1)//2; the total cost is the sum over digits.
We wish to find one square with minimal cost and one with maximal cost.

In [3]:
from ortools.sat.python import cp_model
import math

N = 6

def get_allowed_tuples_by_sum(N):
    lower = 10 ** (N - 1)
    upper = 10 ** N
    sieve = [True] * upper
    sieve[0] = sieve[1] = False
    for i in range(2, int(upper ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, upper, i):
                sieve[j] = False
    allowed = {s: [] for s in range(9 * N + 1)}
    for num in range(lower, upper):
        if sieve[num]:
            digits = tuple(map(int, str(num)))
            s = sum(digits)
            allowed[s].append(digits)
    return allowed

def compute_cost(grid, N):
    numbers = []
    # Rows:
    for i in range(N):
        num = int("".join(str(grid[i][j]) for j in range(N)))
        numbers.append(num)
    # Columns:
    for j in range(N):
        num = int("".join(str(grid[i][j]) for i in range(N)))
        numbers.append(num)
    # Diagonals:
    num_diag1 = int("".join(str(grid[i][i]) for i in range(N)))
    num_diag2 = int("".join(str(grid[i][N - 1 - i]) for i in range(N)))
    numbers.append(num_diag1)
    numbers.append(num_diag2)

    # Use unique primes (even if a prime appears more than once, count it once)
    unique_numbers = set(numbers)

    digit_counts = {str(d): 0 for d in range(10)}
    for num in unique_numbers:
        for d in str(num):
            digit_counts[d] += 1
    cost = sum(count * (count - 1) // 2 for count in digit_counts.values())
    return cost, unique_numbers, digit_counts

best_min_cost = None
best_min_solution = None
best_min_A = None

best_max_cost = None
best_max_solution = None
best_max_A = None

class GridSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, grid, A_fixed):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._grid = grid  # N×N list of cp_model.IntVar
        self._A = A_fixed
        self._solution_count = 0

    def on_solution_callback(self):
        global best_min_cost, best_min_solution, best_min_A
        global best_max_cost, best_max_solution, best_max_A

        self._solution_count += 1
        # Extract the grid solution.
        grid_sol = [[self.Value(self._grid[i][j]) for j in range(N)] for i in range(N)]
        cost, primes_set, digit_counts = compute_cost(grid_sol, N)

        if best_min_cost is None or cost < best_min_cost:
            best_min_cost = cost
            best_min_solution = [row[:] for row in grid_sol]
            best_min_A = self.Value(self._A)

        if best_max_cost is None or cost > best_max_cost:
            best_max_cost = cost
            best_max_solution = [row[:] for row in grid_sol]
            best_max_A = self.Value(self._A)

def main():
    global best_min_cost, best_min_solution, best_min_A
    global best_max_cost, best_max_solution, best_max_A

    print(f"Precomputing allowed tuples for {N}-digit primes...")
    allowed_by_sum = get_allowed_tuples_by_sum(N)

    # Iterate over possible magic sums A (from 0 to 9*N).
    for A_val in range(0, 9 * N + 1):
        if not allowed_by_sum[A_val]:
            continue
        print(f"\nTrying A = {A_val} with {len(allowed_by_sum[A_val])} allowed tuples")

        model = cp_model.CpModel()

        grid = [[model.NewIntVar(0, 9, f"cell_{i}_{j}") for j in range(N)]
                for i in range(N)]

        A_fixed = model.NewIntVar(A_val, A_val, "A")

        for i in range(N):
            model.AddAllowedAssignments(grid[i], allowed_by_sum[A_val])

        for j in range(N):
            col = [grid[i][j] for i in range(N)]
            model.AddAllowedAssignments(col, allowed_by_sum[A_val])

        # Diagonals.
        diag1 = [grid[i][i] for i in range(N)]
        diag2 = [grid[i][N - 1 - i] for i in range(N)]
        model.AddAllowedAssignments(diag1, allowed_by_sum[A_val])
        model.AddAllowedAssignments(diag2, allowed_by_sum[A_val])

        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 30.0

        solution_printer = GridSolutionPrinter(grid, A_fixed)
        status = solver.SearchForAllSolutions(model, solution_printer)
        print(f"A = {A_val} produced {solution_printer._solution_count} solution(s).")

    if best_min_solution is not None:
        print("\nMinimal cost solution:")
        print(f"Cost: {best_min_cost}  (A = {best_min_A})")
        for row in best_min_solution:
            print(" ".join(map(str, row)))
    else:
        print("No solution found for minimal cost.")

    if best_max_solution is not None:
        print("\nMaximal cost solution:")
        print(f"Cost: {best_max_cost}  (A = {best_max_A})")
        for row in best_max_solution:
            print(" ".join(map(str, row)))
    else:
        print("No solution found for maximal cost.")

if __name__ == "__main__":
    main()


Precomputing allowed tuples for 6-digit primes...

Trying A = 4 with 2 allowed tuples
A = 4 produced 0 solution(s).

Trying A = 5 with 14 allowed tuples
A = 5 produced 0 solution(s).

Trying A = 7 with 58 allowed tuples
A = 7 produced 0 solution(s).

Trying A = 8 with 76 allowed tuples
A = 8 produced 0 solution(s).

Trying A = 10 with 204 allowed tuples
A = 10 produced 0 solution(s).

Trying A = 11 with 389 allowed tuples
A = 11 produced 0 solution(s).

Trying A = 13 with 660 allowed tuples
A = 13 produced 0 solution(s).

Trying A = 14 with 852 allowed tuples
A = 14 produced 71 solution(s).

Trying A = 16 with 1448 allowed tuples
A = 16 produced 77 solution(s).

Trying A = 17 with 1971 allowed tuples
A = 17 produced 0 solution(s).

Trying A = 19 with 2832 allowed tuples
A = 19 produced 0 solution(s).

Trying A = 20 with 3101 allowed tuples
A = 20 produced 46 solution(s).

Trying A = 22 with 4064 allowed tuples
A = 22 produced 31 solution(s).

Trying A = 23 with 4651 allowed tuples
A = 